In [1]:
import face_recognition as fr
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import pandas as pd
import sklearn
import math
import pickle #Estudar sobre
from flask import Flask, render_template, Response
print("Import sem erros")


def resizeImage(image, **kwargs):
    if 'scale' in kwargs.keys():
        scale = kwargs.get('scale')
        x = int(image.shape[0] * scale)
        y = int(image.shape[1] * scale)
    return cv2.resize(image, dsize=(x, y), interpolation=cv2.INTER_AREA)

def compare_images(im1, im2):
    plt.Figure(figsize=(10, 10))
    plt.subplot(121)
    plt.imshow(im1)
    plt.subplot(122)
    plt.imshow(im2)

def load_image(path, gray=True):
    im = cv2.imread(path)
    if gray: im = cv2.cv2tColor(im, cv2.COLOR_BGR2RGB)
    return im

def flat_images(images):
    #Flatten the images array
    n, m = images[0].shape
    k = len(images)
    flat_images = np.zeros((n * m, k), dtype='uint8').T
   
    for i in range(k):
        flat_images[i] = images[i].flatten()
    return flat_images


def plotSample(images, labels):
    size = len(images)
    plot_size = np.array(list((3,4))) * 4
    plt.figure(figsize=tuple(plot_size))
    for i in range(12):
        plt.subplot(3,4, i+1)
        plt.imshow(images[i])
        plt.yticks([])
        plt.xticks([])
        plt.xlabel(labels[i])
    plt.show()
   
#Lmebrar que esta recebe a predição com distância = True
#Errado
def get_labels(labels, knn_pred):
   names = []
   index_pred = [i for i in knn_pred[1][0]]
   for i in index_pred:
       names.append(labels[i])
   return names

def read_dir(directory, for_one=False, retrieve_one_image=False):
    path = []
    images = []
    data = []
    
    if not(for_one):
        for sub_folder in os.listdir(directory):
            for filename in os.listdir(os.path.join(directory, sub_folder)):
                image_path = directory+'/'+sub_folder+'/'+filename
                data.append({"Name": sub_folder, 'Path': image_path})
                if retrieve_one_image:
                    break
    else:
        for image in os.listdir(directory):
            image_path = os.path.join(directory,image)
            data.append({"Name": directory.split('/').pop(), "Path":image_path})            
     
    df = pd.DataFrame(data)
    return df

def read_image(path):
    images = []
    for x in path:
        im = cv2.imread(x)
        im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
        images.append(im)
    return images


def knn_train(X_train, y_train, model_save_path=None, threads=-1):
    from sklearn.neighbors import KNeighborsClassifier
    n_neighbors = int(math.sqrt(len(X_train))) #Sera susbtituído pelo elbow method
    
    model = KNeighborsClassifier(n_neighbors= n_neighbors, weights='distance', algorithm='auto', p=2, metric='minkowski', n_jobs = threads)
    model.fit(X_train, y_train)
    if model_save_path:
        save_binary(model, model_save_path)
    
    return model

def knn_predict_(image, model_path=None, verbose=False):
    if model_path:
        with open(model_path, 'rb') as f:
            model = pickle.load(f)
   
    y_pred = model.kneighbors(image, n_neighbors=1)
   
    return y_pred

def save_binary(model, path):
    with open(path, 'wb') as f:
        pickle.dump(model, f)

def load_binary(path):
    with open(path, 'rb') as f:
        model = pickle.load(f)
    return model

def unpaking_array(*args):
    new_list = list
    
    
def first_train(train_dir, model_save_path):
    df = read_dir(train_dir, retrieve_one_image=True)
    df = df.iloc[:1500, :] #Here because of the Dataset 
    df['Image'] = read_image(df['Path'])
    print(df.head(10))

    df["Face Location"] = df["Image"].apply(lambda x: fr.face_locations(x, 2, model="hog"))
    df["Face Encoding"] = df.apply(lambda x: fr.face_encodings(x["Image"], x["Face Location"]), axis=1)
    print(df.head(10))

    # model = knn_train([x[0] for x in df['Face Encoding']], df['Name'].values, model_save_path)
    model = [x for x in df['Face Encoding']]
    return model, df

    


Import sem erros


In [3]:
# Realizando leitura do dataset

# if __name__ == '__main__':
model_save_path = "./knn_model.clf"
train_dir = "archive/lfw-deepfunneled"
df_save_path = 'bkp/dataset_example.pkl'


# model, df = first_train(train_dir, model_save_path)


In [13]:

df = read_dir(train_dir, retrieve_one_image=True)
df = df.iloc[:1500, :] #Here because of the Dataset 
df['Image'] = read_image(df['Path'])
print(df.head(10))

                Name                                               Path  \
0        Hugo_Colace  archive/lfw-deepfunneled/Hugo_Colace/Hugo_Cola...   
1      Alexis_Bledel  archive/lfw-deepfunneled/Alexis_Bledel/Alexis_...   
2        Jim_Schwarz  archive/lfw-deepfunneled/Jim_Schwarz/Jim_Schwa...   
3  Patti_Balgojevich  archive/lfw-deepfunneled/Patti_Balgojevich/Pat...   
4  Yekaterina_Guseva  archive/lfw-deepfunneled/Yekaterina_Guseva/Yek...   
5        Bruce_Arena  archive/lfw-deepfunneled/Bruce_Arena/Bruce_Are...   
6      Harland_Braun  archive/lfw-deepfunneled/Harland_Braun/Harland...   
7         Ken_Wharfe  archive/lfw-deepfunneled/Ken_Wharfe/Ken_Wharfe...   
8    Anatoliy_Kinakh  archive/lfw-deepfunneled/Anatoliy_Kinakh/Anato...   
9    Ronald_Young_Jr  archive/lfw-deepfunneled/Ronald_Young_Jr/Ronal...   

                                               Image  
0  [[[143, 137, 87], [143, 137, 87], [144, 138, 8...  
1  [[[77, 46, 51], [76, 45, 50], [74, 45, 49], [7...  
2  [[[0, 

In [21]:
df["Face Location"] = df["Image"].apply(lambda x: fr.face_locations(x, 2, model="hog"))
df["Face Encoding"] = df.apply(lambda x: fr.face_encodings(x["Image"], x["Face Location"]), axis=1)
df.to_pickle(df_save_path) #parquet does not support 2D arrays
print("Fim encode")


Fim encode


In [40]:
df.to_pickle(df_save_path)

In [4]:
df = pd.read_pickle(df_save_path)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Name           1500 non-null   object
 1   Path           1500 non-null   object
 2   Image          1500 non-null   object
 3   Face Location  1500 non-null   object
 4   Face Encoding  1500 non-null   object
dtypes: object(5)
memory usage: 58.7+ KB


In [61]:
df_2 = df.copy()
df_2.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Name           1500 non-null   object
 1   Path           1500 non-null   object
 2   Image          1500 non-null   object
 3   Face Location  1500 non-null   object
 4   Face Encoding  1500 non-null   object
dtypes: object(5)
memory usage: 58.7+ KB


In [66]:
kk = df['Face Location'].iterrows()
for item in kk:
    print(item)

AttributeError: 'Series' object has no attribute 'iterrows'

In [75]:
for x, y in zip(df['Face Location'][:13], df['Face Encoding'][:13]):
    if len(x) > 1:
        list(x)
        for x1, y1 in zip(x, y):
            print(sum(x1))
            print(y1)


588
[-1.40803248e-01  7.86165968e-02  6.62570866e-03 -5.87223396e-02
 -6.92278966e-02 -3.82894091e-02 -4.40395735e-02 -1.10197149e-01
  1.34513855e-01  1.57549027e-02  1.39137194e-01 -7.28460625e-02
 -2.57932097e-01  4.54097055e-04 -1.16960108e-01  9.59389806e-02
 -1.55617461e-01 -1.09792896e-01 -1.87494308e-02 -1.95994899e-02
  5.90403080e-02  5.39695919e-02 -5.16973436e-02  6.09443113e-02
 -1.04232766e-01 -2.80124128e-01 -1.05995528e-01 -7.78064728e-02
  2.04362161e-02 -8.11036453e-02 -2.84332763e-02  7.34889731e-02
 -1.92726180e-01 -6.32749423e-02  3.74213383e-02  1.76160544e-01
 -2.65048984e-02  1.24252848e-02  1.15373716e-01  1.07576549e-01
 -8.89127329e-02 -4.36579734e-02  8.35225806e-02  3.84131819e-01
  1.38959259e-01  5.27912863e-02 -8.18355195e-03  3.93852405e-03
  3.67053114e-02 -2.85501212e-01  1.07814036e-01  1.58008724e-01
  1.53121248e-01  1.13862582e-01  1.29191548e-01 -2.19440624e-01
  2.92721763e-02  1.18769661e-01 -1.81706145e-01  1.63930342e-01
  7.66438916e-02  1.1

In [83]:
soma = lambda x: sum(x)
sum_list_loc_m = []
for x, y in zip(df_2['Face Location'][:13], df_2['Face Encoding'][:13]):
    if len(x) > 1:
        list_locations_m = list(x)
        for loc in list_locations_m:
            sum_list_loc_m.append(loc)
        sum_list_loc_m.sort(key=soma, reverse=True)
        print(sum_list_loc_m)
        

[(57, 237, 134, 160), (89, 171, 182, 78), (109, 74, 186, 0)]


In [143]:
sum_list_loc_m = []
result = zip(df_2['Face Location'][:13], df_2['Face Encoding'][:13])
for face_location, face_encoding in result:
    if len(face_location) > 1:
        mydict = dict(zip(face_location, face_encoding))
        sorted_dict = dict()
        for key in sorted(mydict, key=lambda x: sum(x), reverse=True):
            sorted_dict.update({key: mydict[key]})

first = next(iter(mydict_2.items()))
print(first)
x, y = first
print(x)
print(y)
  
        # res = zip(face_location, face_encoding)
        # res_set = dict(res)
        # for x, y in res_set:
        #     print(x)
        #     print(y)
        # sum_list_loc_m = list(face_location)
        # sum_list_loc_m.sort(key=lambda x: sum(x), reverse=True)
        # print(sum_list_loc_m)

        

        

((57, 237, 134, 160), array([-1.40803248e-01,  7.86165968e-02,  6.62570866e-03, -5.87223396e-02,
       -6.92278966e-02, -3.82894091e-02, -4.40395735e-02, -1.10197149e-01,
        1.34513855e-01,  1.57549027e-02,  1.39137194e-01, -7.28460625e-02,
       -2.57932097e-01,  4.54097055e-04, -1.16960108e-01,  9.59389806e-02,
       -1.55617461e-01, -1.09792896e-01, -1.87494308e-02, -1.95994899e-02,
        5.90403080e-02,  5.39695919e-02, -5.16973436e-02,  6.09443113e-02,
       -1.04232766e-01, -2.80124128e-01, -1.05995528e-01, -7.78064728e-02,
        2.04362161e-02, -8.11036453e-02, -2.84332763e-02,  7.34889731e-02,
       -1.92726180e-01, -6.32749423e-02,  3.74213383e-02,  1.76160544e-01,
       -2.65048984e-02,  1.24252848e-02,  1.15373716e-01,  1.07576549e-01,
       -8.89127329e-02, -4.36579734e-02,  8.35225806e-02,  3.84131819e-01,
        1.38959259e-01,  5.27912863e-02, -8.18355195e-03,  3.93852405e-03,
        3.67053114e-02, -2.85501212e-01,  1.07814036e-01,  1.58008724e-01,
   

In [86]:
# sub_frame = df_2.loc[lambda x: x['Face Location'] > 1]
kk = df_2['Face Location'].filter(lambda x: len(x) > 1)
kk

TypeError: 'function' object is not iterable

In [58]:
df.iloc[12]
test_m = zip(df['Face Location'][12], df['Face Encoding'][12])
test_o = zip(df['Face Location'][13], df['Face Encoding'][13])
# este são os valores recebidos continuamente pelo serão zipados

In [59]:
len_zip = sum(1 for _ in test_m)
if len_zip > 1:
    encodings = []
    for location, encoding in test_m:
        encodings.append(encoding)
        print(location)

    print(encodings)
    # y_pred = model.predict(encodings)
        

[]


In [51]:
for loc, enc in test_m:
        print(loc)
        print(enc)

In [53]:
test_m

In [ ]:
#Devo retornar um array com as posições de cada face e seus nomes para que o draw possa iterar neste, mesmo que seja unitário
#No momento do cadastro a screenshot deverá ser validada, para que se armazene somente a foto que houver apenas uma face
model = knn_train()...
y_pred = model.predict(frame_encoded)
candidate = df[df['Name'] == y_pred]
face_distance = can
